In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import numba as nb
import cython as cy
import scipy
import const
from scipy.special import gamma
from scipy.special import kv
from scipy import *
from scipy.integrate import odeint
import math as m

In [ ]:
def min(a, b):
    if a < b:
        return a
    else:
        return b

#TODO: Find temp true value
temp = 1e6
    
xi = const.m_p * const.c**2 / (const.k_B * temp)

In [ ]:
#Définition des fonctions de Bessel de première espèce (avec une limite finie en 0)

def K1(x) :
    return kv(1, x)

def K2(x) :
    return kv(2, x)

def K3(x) :
    return kv(3, x)

def gamma_maj(xi):
    return (K3(xi)/K2(xi) - 1/xi)

def gamma_chapeau_prime(xi):
    a = xi*(3*K3(xi) + K1(xi) - 4*K2(xi))
    return 1 + 4*K2(xi)/a





In [ ]:
tetav=5
phi=5
c=3*10**8
mp = 1.67e-27
Gamma_sh_max=100
teta=5
E=10**44
n0=10**(6)   #let's say 1 particle by cubic cm
mu=m.sin(teta)*m.sin(tetav)*m.cos(phi)+m.cos(teta)*m.cos(tetav)


def C_BM(E, n0, mp, c):
    return np.sqrt(17 * E / (8 * np.pi * n0 * mp * c**5))

def C_ST(E, n0, mp, c):
    return (2/5) * 1.15 * (E / (n0 * mp * c**5))**(1/5)

def radius(T, mu, c):
    def f(R, t):
        return T + mu * R / c

    Rp = 0
    Rs = 1
    while abs(Rp - Rs) > 10 ** (-2):
        sol = odeint(f, [0, Rs], [0, T])
        Rp = Rs
        Rs = sol[0][1]
    return Rs

    
def shock_speed(C_BM, C_ST, Gamma_sh_max,t,T):
    t=T+mu*radius(t,mu,c)/c
    sh_max = Gamma_sh_max
    mine=np.minimum((C_BM(E, n0, mp, c)**2) * (t)**(-3) + (C_ST(E, n0, mp, c)**2) * (t)**(-6/5), sh_max**2)
    beta_sh=np.sqrt(mine/(1+mine))
    return beta_sh

C_BM_val = C_BM(E, n0, mp, c)
C_ST_val = C_ST(E, n0, mp, c)

beta_sh = shock_speed(C_BM_val, C_ST_val, Gamma_sh_max, 10**5, 10**5)

beta_u = beta_sh
beta = np.sqrt(1 - 1/(gamma_maj(xi) ** 2))
beta_d = (beta_sh - beta) / (1 - beta * beta_sh)

zeta = const.m_p * const.c**2/(const.k_B*T_prime)
gamma_maj = K3(zeta) /K2(zeta) - 1/zeta
gamma_chapeau_prime = 1 + 4*K2(zeta)/(zeta*(3*K3(zeta) + K1(zeta) - 4*K2(zeta)))
gamma_sh = sqrt((gamma_maj+1)/(2+gamma_chapeau_prime*(2-gamma_chapeau_prime)*(gamma_maj-1)))*(1+gamma_chapeau_prime*(gamma_maj-1))

In [ ]:
#Pour la fonction d'émission on renormalise tout:
epsi_b_gaussianJ = 2.44*(1/np.power(10,4))
epsi_b_hollow_coneJ = 4.12*(1/np.power(10,5))
epsi_b_SpindleJ = 6.60*(1/np.power(10,4))
#pour l'instant en première approximation on prend p = 2.05


def power_I(x,y):
    return 1/np.power(x,y)

#Rapport de comparaison pour les refroidissemts entre nu_prim_c et nu_prim_m
def Kappa(n0,epsilon_B,t,gamma_sh):
    return nu_prime_c(n0, epsilon_B, gamma_sh,t)/nu_prime_m(epsilon_B, gamma_sh, n0)

def p(beta_u,beta_d):
    a = 3*beta_u - 2*beta_u*(beta_d**2) + beta_d**3
    b = beta_u - beta_d
    return a/b - 2

def n_prime_r(n0,gamma_sh, beta_u,beta_d):
    return n_prime(gamma_sh,n0)*min(1,((p(beta_u, beta_d) - 2)*const.m_p/(p(beta_u,beta_d) - 1)
                                       *const.m_e)*const.epsi_e*(gamma_maj(sol(gamma_sh)) - 1))

def FastcEmission(n0,x, Kappa,p,epsilon_B,t,gamma_sh):
    if x > Kappa(n0,epsilon_B,t,gamma_sh):
        return np.power(Kappa(n0,epsilon_B,t,gamma_sh),-1/3)*np.power(x,1/3)
    elif x >= Kappa(n0,epsilon_B,t,gamma_sh) and x <1:
        return np.power(Kappa(n0,epsilon_B,t,gamma_sh),1/2)*power_I(x,-1/2)
    else:
        power_I(Kappa(n0,epsilon_B,t,gamma_sh),-1/2)*power_I(x,-p/2)

#Cas du slow-cooling
def SlowcEmission(x,Kappa,p,epsilon_B,t,gamma_sh,n0):
    if x < 1:
        return np.power(x,1/3)
    elif x>=1 and x < Kappa(n0,epsilon_B,t,gamma_sh):
        return power_I(x,-(p-1)/2)
    else:
        return power_I(Kappa(n0,epsilon_B,t,gamma_sh),-1/2)*power_I(-p/2)




#Fonction d'émission
def Emission_function(n0,Kappa,x,epsilon_B,t,gamma_sh,p):
    if Kappa(n0,epsilon_B,t,gamma_sh) > 1:
        return SlowcEmission(x,Kappa,p,epsilon_B,t,gamma_sh,n0)
    else:
        return FastcEmission(n0,x, Kappa,p,epsilon_B,t,gamma_sh)


def fact_denorm(beta_u,beta_d,n0,gamma_sh,epsilon_b):
    return 0.88*(256*(p(beta_u,beta_d) - 1)*(const.q_e**3)*n_prime_r(n0,gamma_sh,beta_u,beta_d)
                 *B_prime(epsilon_b,gamma_sh,n0)/27*(3*(p(beta_u,beta_d) - 1)*const.m_e*(const.celer**2)))



In [ ]:
# Détermination de Ksi, car il est défini implicitement ici. 
#Find the rpoblem with ksi ici
from scipy.optimize import fsolve
def eq(gamma_sh,xi):
    a = (gamma_maj(xi) + 1)*(1 + gamma_chapeau_prime(xi)*(gamma_maj(xi) - 1))**2
    b = 2 + gamma_chapeau_prime(xi)*(2 - gamma_chapeau_prime(xi))*(gamma_maj(xi) - 1)
    return gamma_sh**2 - a/b

def sol(gamma_sh):
    def func(xi):
        return eq(gamma_sh, xi)
    solve = fsolve(func,1)
    return solve[0]

sol_vect = np.vectorize(sol)
g_m = np.vectorize(gamma_maj)
g_c = np.vectorize(gamma_chapeau_prime)

def n_prime(gamma_sh, n0):
    return 4*gamma_maj(sol(gamma_sh))*n0
def e_i_p(n0,gamma_sh):
    return const.m_p*((const.celer)**2)(gamma_maj(sol(gamma_sh)) - 1)*n_prime(n0,gamma_sh)

epsi_b = 4.12*(1/np.power(10,5))
def gamma_prime_c(n0,epsilon_B, gamma_sh,t):
    return (3 * const.m_e * const.celer* gamma_maj(sol(gamma_sh))) / (4 * const.sigma_T * epsilon_B *e_i_p(gamma_sh) * t*n0)

def gamma_prime_m(p, gamma_sh):
    arg = ((p - 2)*const.m_p/(p - 1)*const.m_e)*const.epsi_e*(gamma_maj(sol(gamma_sh)) - 1)
    return np.max(1, arg)

def B_prime(epsilonB, gamma_sh,n0):
    return (np.sqrt(8*const.pi*epsilonB*e_i_p(gamma_maj,gamma_chapeau_prime, gamma_sh,n0)))

def nu_prime_m(epsilon_B, gamma_sh,n0,p ):
    return 3/16 * gamma_prime_m(p,gamma_sh)**2 * const.q_e*B_prime(epsilon_B, gamma_sh,n0) / (const.m_e * const.celer)

def nu_prime_c(n0,epsilon_B, t, gamma_sh):
    return 3/16 * gamma_prime_c(n0,epsilon_B,gamma_sh, t)**2 * const.q_e *B_prime(epsilon_B, gamma_sh, n0) / (const.m_e * const.celer)


In [ ]:

#TODO: implement p_tildt when y'm  < y'c
def p_tildt(gamma_prime_c, gamma_prime_m): 
    if gamma_prime_m >= gamma_prime_c:
        return 2 
    else:
        return (3 * beta_u * - 2 * beta_u * beta_d ** 2 + beta_d ** 3) / (beta_u - beta_d) - 2

#TODO: n'R à rajouter, et b_prime  
def c_11(nu_prime, nu_prime_c, nu_prime_m, gamma_prime_m, gamma_prime_c):
    p_tildt = p_tildt(gamma_prime_c, gamma_prime_m)
    nu_min = min(nu_prime_m, nu_prime_c)
    gamma_min = min(gamma_prime_m, gamma_prime_c)
    return 2 ** 6 *const.pi ** (5/6) * (p_tildt + 2) * (p_tildt - 1) * const.q_e * n_prime_r / (15 * (3*p_tildt + 2) * gamma(5 / 6) * gamma_min ** 5 * b_prime) * (nu_prime / nu_min) ** (-5/3)

#TODO: n'R et b_prime à rajouter
def c_14(nu_prime, nu_prime_c, nu_prime_m, gamma_prime_m, gamma_prime_c):
    p_tildt = p_tildt(gamma_prime_c, gamma_prime_m)
    nu_min = min(nu_prime_m, nu_prime_c)
    gamma_min = min(gamma_prime_m, gamma_prime_c)
    return (2 ** ((3*p_tildt + 8) / 2) * (p_tildt - 1) * gamma(3/2 + p_tildt/4) * gamma(1/6 + p_tildt/4) * const.q_e * n_prime_r ) / (3**(3/2) * const.pi **((p_tildt + 1 )/2) * gamma(2 + p_tildt/4) * gamma_min ** 5 *b_prime) *(nu_prime / nu_min) ** (-(p_tildt + 4) / 2)

def nu_prime_zero(nu_prime_min): 
    return ((5*(3 * p_tildt +2) * gamma(3/2 + p_tildt/4) *gamma(11/6 + p_tildt/4) * gamma(1/6 + p_tildt/4) * gamma(5/6)) / (p_tildt + 2) * gamma(2 + p_tildt/4))** (6 / (3*p_tildt +2)) * ((2**( 3 * (3 * p_tildt -4))) / (27 * const.pi ** (3*p_tildt + 8) )) ** (1 / (3*p_tildt + 2)) * nu_prime_min

def alpha_prime_nu_prime(nu_prime, nu_prime_c, nu_prime_m, gamma_prime_m, gamma_prime_c):
    p_tildt = p_tildt(gamma_prime_c, gamma_prime_m)
    nu_min = min(nu_prime_m, nu_prime_c)
    nu_zero = nu_prime_zero(nu_min)
    if nu_prime < nu_zero:
        alpha_prime_nu_prime_zero = c_11(nu_zero, nu_prime_c, nu_prime_m, gamma_prime_m, gamma_prime_c)
        eta =  -5/3
    else : 
        alpha_prime_nu_prime_zero = c_14(nu_zero, nu_prime_c, nu_prime_m, gamma_prime_m, gamma_prime_c)
        eta = -(p_tildt + 4) / 2
    return alpha_prime_nu_prime_zero * (nu_prime / nu_zero) ** eta
#TODO: improt gamma_maj, find value of beta and mu

def nu_prime(nu, xi, beta, mu):
    return gamma_maj(xi) * (1 - beta * mu) * nu 


'''
Plotting phase : 
'''
#Plotting with gamma_sh as parameter knowing that xi is a function of gamma_sh

gamma_sh = np.linspace(1.2, 100, 100)

In [ ]:
zeta = mp * c**2/(kB*T_prime)
gamma_maj = K3(zeta) /K2(zeta) - 1/zeta
gamma_chapeau_prime = 1 + 4*K2(zeta)/(zeta*(3*K3(zeta) + K1(zeta) - 4*K2(zeta)))
print(zeta, gamma_maj, gamma_chapeau_prime)

alpha_nu = gamma_maj*(1-beta*mu)*alpha_prime_nu_prime
delta_s = R/(12*np.abs(mu - beta_sh)*gamma_maj**2)
tau_nu = alpha_nu*delta_s